![JohnSnowLabs](https://sparknlp.org/assets/images/logo.png)

[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/JohnSnowLabs/spark-nlp/blob/master/examples/python/transformers/openvino/HuggingFace_OpenVINO_in_Spark_NLP_BertForMultipleChoice.ipynb)

# Import OpenVINO BertForMultipleChoice models from HuggingFace 🤗 into Spark NLP 🚀

This notebook provides a detailed walkthrough on optimizing and exporting BertForMultipleChoice  models from HuggingFace for use in Spark NLP, leveraging the various tools provided in the [Intel OpenVINO toolkit](https://www.intel.com/content/www/us/en/developer/tools/openvino-toolkit/overview.html) ecosystem.

Let's keep in mind a few things before we start 😊

- OpenVINO support was introduced in  `Spark NLP 5.4.0`, enabling high performance inference for models. Please make sure you have upgraded to the latest Spark NLP release.
- You can import models for BertForMultipleChoice from BertForMultipleChoice  and they have to be in `For Multiple Choice` category.

## 1. Export and Save the HuggingFace model

- Install required packages for ONNX export and OpenVINO conversion:




In [1]:
!pip install --quiet onnx==1.18.0 onnxruntime==1.22.0 optimum[onnxruntime]==1.26.1 openvino==2025.2.0

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 17.6/17.6 MB 27.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 16.4/16.4 MB 20.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 424.6/424.6 kB 13.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 47.6/47.6 MB 7.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 3.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 68.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 42.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 40.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 2.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 5.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 8.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 10.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2

- Locked versions to avoid breaking changes.
- These work together for exporting Hugging Face models to ONNX and converting to OpenVINO IR.

[Optimum Intel](https://github.com/huggingface/optimum-intel?tab=readme-ov-file#openvino) is the interface between the Transformers library and the various model optimization and acceleration tools provided by Intel. HuggingFace models loaded with optimum-intel are automatically optimized for OpenVINO, while being compatible with the Transformers API.
- Normally, to load a HuggingFace model directly for inference/export, just replace the `AutoModelForXxx` class with the corresponding `OVModelForXxx` class. However, ForMultipleChoice is not yet available so we will use `openvino.convert_model()` after exporting ONNX model
- We'll use [irfanamal/bert_multiple_choice](https://huggingface.co/irfanamal/bert_multiple_choice) model from HuggingFace as an example
- We also need the `vocab.txt` saved from `AutoTokenizer`. This is the same for every model, these are assets (saved in `/assets`) needed for tokenization inside Spark NLP.

In [ ]:
from optimum.onnxruntime import ORTModelForMultipleChoice
from transformers import BertTokenizer
import openvino as ov

MODEL_NAME = "irfanamal/bert_multiple_choice"
ONNX_MODEL_PATH = "ov_models/bert_multiple_choice"
OPENVINO_MODEL_PATH = "openvino_models"

ort_model = ORTModelForMultipleChoice.from_pretrained(MODEL_NAME, export=True)
ort_model.save_pretrained(ONNX_MODEL_PATH)

/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


config.json:   0%|          | 0.00/670 [00:00<?, ?B/s]

pytorch_model.bin:   0%|          | 0.00/438M [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/314 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/711k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/125 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/438M [00:00<?, ?B/s]

In [4]:
ov_model = ov.convert_model("onnx_models/bert_multiple_choice/model.onnx")
ov.save_model(ov_model, "openvino_models/openvino_model.xml")

tokenizer = BertTokenizer.from_pretrained(MODEL_NAME)
tokenizer.save_pretrained("openvino_models")

('openvino_models/tokenizer_config.json',
 'openvino_models/special_tokens_map.json',
 'openvino_models/vocab.txt',
 'openvino_models/added_tokens.json')

Let's have a look inside these two directories and see what we are dealing with:

In [5]:
!ls -lR {ONNX_MODEL_PATH}

onnx_models/bert_multiple_choice:
total 427972
-rw-r--r-- 1 root root       630 Jun 19 22:31 config.json
-rw-r--r-- 1 root root 438238205 Jun 19 22:31 model.onnx


In [6]:
!ls -lR {OPENVINO_MODEL_PATH}

openvino_models:
total 214656
-rw-r--r-- 1 root root 218970320 Jun 19 22:31 openvino_model.bin
-rw-r--r-- 1 root root    591013 Jun 19 22:31 openvino_model.xml
-rw-r--r-- 1 root root       695 Jun 19 22:31 special_tokens_map.json
-rw-r--r-- 1 root root      1272 Jun 19 22:31 tokenizer_config.json
-rw-r--r-- 1 root root    231508 Jun 19 22:31 vocab.txt


- We need to move `vocab.txt` from the tokenizer to `assets` folder which Spark NLP will look for

In [7]:
!mkdir {OPENVINO_MODEL_PATH}/assets && mv {OPENVINO_MODEL_PATH}/vocab.txt {OPENVINO_MODEL_PATH}/assets

- Voila! We have our `vocab.txt` inside assets directory

## Import and Save BertForMultipleChoice in Spark NLP


- Install and set up Spark NLP in Google Colab
- This example uses specific versions of `pyspark` and `spark-nlp` that have been tested with the transformer model to ensure everything runs smoothly.

In [8]:
!pip install -q pyspark==3.5.4 spark-nlp==5.5.3

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 317.3/317.3 MB 4.2 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 635.7/635.7 kB 36.4 MB/s eta 0:00:00


Let's start Spark with Spark NLP included via our simple `start()` function

In [9]:
import sparknlp

spark = sparknlp.start()

print("Spark NLP version: ", sparknlp.version())
print("Apache Spark version: ", spark.version)

Spark NLP version:  5.5.3
Apache Spark version:  3.5.4


- Let's use `loadSavedModel` functon in `BertForMultipleChoice` which allows us to load TensorFlow model in SavedModel format
- Most params can be set later when you are loading this model in `BertForMultipleChoice` in runtime, so don't worry what you are setting them now
- `loadSavedModel` accepts two params, first is the path to the TF SavedModel. The second is the SparkSession that is `spark` variable we previously started via `sparknlp.start()`
- NOTE: `loadSavedModel` accepts local paths in addition to distributed file systems such as `HDFS`, `S3`, `DBFS`, etc. This feature was introduced in Spark NLP 4.2.2 release. Keep in mind the best and recommended way to move/share/reuse Spark NLP models is to use `write.save` so you can use `.load()` from any file systems natively.


In [10]:
from sparknlp.annotator import BertForMultipleChoice

bertMultpleChoiceClassifier = BertForMultipleChoice.loadSavedModel(
     f"{OPENVINO_MODEL_PATH}",
     spark
 )\
  .setInputCols(["document_question", "document_context"])\
  .setOutputCol("answer")

- Let's save it on disk so it is easier to be moved around and also be used later via `.load` function

In [11]:
bertMultpleChoiceClassifier.write().overwrite().save("./{}_spark_nlp_openvino".format(MODEL_NAME))

Let's clean up stuff we don't need anymore

In [12]:
!rm -rf {ONNX_MODEL_PATH}
!rm -rf {OPENVINO_MODEL_PATH}

Awesome 😎  !

This is your`BertForMultipleChoice` model from HuggingFace 🤗  loaded and saved by Spark NLP 🚀

In [13]:
! ls -l {MODEL_NAME}_spark_nlp_openvino

total 214464
-rw-r--r-- 1 root root 219595121 Jun 19 22:33 bert_mc_classification_openvino
drwxr-xr-x 3 root root      4096 Jun 19 22:33 fields
drwxr-xr-x 2 root root      4096 Jun 19 22:33 metadata


Now let's see how we can use it on other machines, clusters, or any place you wish to use your new and shiny `BertForMultipleChoice` model 😊

In [14]:
testing_data = [
    ("In Italy, pizza in formal settings is served unsliced.", "With fork and knife, In hand."),
    ("The Eiffel Tower is in which country?", "Germany, France, Italy"),
    ("Which animal is the king of the jungle?", "Lion, Elephant, Tiger, Leopard"),
    ("Water boils at what temperature?", "90°C, 120°C, 100°C"),
    ("Which planet is the Red Planet?", "Jupiter, Mars, Venus"),
    ("Primary language in Brazil?", "Spanish, Portuguese, English"),
    ("The Great Wall of China was built to defend against?", "Greeks, Romans, Mongols, Persians"),
    ("Element with symbol 'O'?", "Oxygen, Osmium, Ozone"),
    ("Sahara Desert is in which continent?", "Asia, Africa, South America"),
    ("Who painted the Mona Lisa?", "Van Gogh, Da Vinci, Picasso")
]

In [15]:
testing_df = spark.createDataFrame(testing_data, ["question", "choices"])
testing_df.show(truncate=False)

+------------------------------------------------------+---------------------------------+
|question                                              |choices                          |
+------------------------------------------------------+---------------------------------+
|In Italy, pizza in formal settings is served unsliced.|With fork and knife, In hand.    |
|The Eiffel Tower is in which country?                 |Germany, France, Italy           |
|Which animal is the king of the jungle?               |Lion, Elephant, Tiger, Leopard   |
|Water boils at what temperature?                      |90°C, 120°C, 100°C               |
|Which planet is the Red Planet?                       |Jupiter, Mars, Venus             |
|Primary language in Brazil?                           |Spanish, Portuguese, English     |
|The Great Wall of China was built to defend against?  |Greeks, Romans, Mongols, Persians|
|Element with symbol 'O'?                              |Oxygen, Osmium, Ozone            |

This is how you can use your loaded classifier model in Spark NLP 🚀 pipeline:

In [16]:
from pyspark.ml import Pipeline
from sparknlp.annotator import BertForMultipleChoice
from sparknlp.base import MultiDocumentAssembler

document_assembler = MultiDocumentAssembler() \
    .setInputCols(["question", "choices"]) \
    .setOutputCols(["document_question", "document_choices"])

bert_model = BertForMultipleChoice() \
    .load(f"{MODEL_NAME}_spark_nlp_openvino") \
    .setInputCols(["document_question", "document_choices"]) \
    .setOutputCol("answer") \
    .setBatchSize(4)

pipeline = Pipeline(stages=[
    document_assembler,
    bert_model
])

pipeline_model = pipeline.fit(testing_df)
result_df = pipeline_model.transform(testing_df)

result_df.select("question", "choices", "answer.result").show(truncate=False)

+------------------------------------------------------+---------------------------------+---------------------+
|question                                              |choices                          |result               |
+------------------------------------------------------+---------------------------------+---------------------+
|In Italy, pizza in formal settings is served unsliced.|With fork and knife, In hand.    |[With fork and knife]|
|The Eiffel Tower is in which country?                 |Germany, France, Italy           |[ France]            |
|Which animal is the king of the jungle?               |Lion, Elephant, Tiger, Leopard   |[Lion]               |
|Water boils at what temperature?                      |90°C, 120°C, 100°C               |[90°C]               |
|Which planet is the Red Planet?                       |Jupiter, Mars, Venus             |[ Mars]              |
|Primary language in Brazil?                           |Spanish, Portuguese, English     |[ Engl

That's it! You can now go wild and use hundreds of `BertForMultipleChoice` models from HuggingFace 🤗 in Spark NLP 🚀
